# Importing the libraries

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
import datetime as dt

# Importing the dataset

## Stock

In [59]:
ticker = 'RELIANCE.NS'

## Timeframe

In [60]:
startyear = 2020
startmonth = 7
startday = 9
start = dt.datetime(startyear, startmonth, startday)

now = dt.datetime.now()
end = now

# endyear = 2019
# endmonth = 10
# endday = 7
# end = dt.datetime(endyear, endmonth, endday)
print(start)
print(end)

2020-07-09 00:00:00
2020-08-09 10:27:03.924324


## Getting the dataset

In [61]:
df = pdr.get_data_yahoo(ticker, start, end)

[*********************100%***********************]  1 of 1 completed


# Algorithm

## List of moving averages used

In [62]:
emasUsed = [3, 5, 8, 10, 12, 15, 30, 35, 40, 45, 50, 60]

## Acquiring EMAs

In [63]:
for x in emasUsed:
    ema = x
    df["EMA_"+str(ema)] = round(df.iloc[:, 4].ewm(span=ema, adjust=False).mean(), 2)

In [64]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,EMA_3,EMA_5,EMA_8,EMA_10,EMA_12,EMA_15,EMA_30,EMA_35,EMA_40,EMA_45,EMA_50,EMA_60
Date,,,,,,,,,,,,,,,,,,
2020-08-03,2051.300049,2057.000000,2003.099976,2009.000000,2009.000000,21559142,2050.17,2065.49,2062.01,2052.67,2041.68,2024.94,1962.61,1948.96,1937.66,1928.17,1920.11,1907.19
2020-08-04,2020.599976,2167.000000,2000.250000,2150.600098,2150.600098,42476932,2100.38,2093.86,2081.69,2070.48,2058.44,2040.65,1974.74,1960.16,1948.04,1937.84,1929.15,1915.17
2020-08-05,2169.000000,2196.000000,2118.300049,2126.449951,2126.449951,34404810,2113.42,2104.72,2091.64,2080.65,2068.90,2051.37,1984.53,1969.40,1956.75,1946.04,1936.89,1922.10
2020-08-06,2157.000000,2167.699951,2107.800049,2134.100098,2134.100098,22762545,2123.76,2114.52,2101.07,2090.37,2078.93,2061.72,1994.18,1978.55,1965.40,1954.22,1944.62,1929.05
2020-08-07,2129.800049,2157.800049,2120.199951,2146.449951,2146.449951,15217200,2135.10,2125.16,2111.16,2100.57,2089.32,2072.31,2004.00,1987.88,1974.23,1962.58,1952.54,1936.18


## Trading calls

In [65]:
pos = 0
num = 0
percentchange = []

for i in df.index:
    cmin = min(df["EMA_3"][i], df["EMA_5"][i], df["EMA_8"][i], df["EMA_10"][i], 
               df["EMA_12"][i], df["EMA_15"][i])
    cmax = max(df["EMA_30"][i], df["EMA_35"][i], df["EMA_40"][i], df["EMA_45"][i], 
                df["EMA_50"][i], df["EMA_60"][i])
    
    close = df["Adj Close"][i]
    
    if(cmin>cmax):
        #print("Red White Blue")
        if(pos == 0):
            bp = close
            pos = 1
            print("Buying now at "+str(bp))
            
    elif(cmin<cmax):
        #print("Blue White Red")
        if(pos == 1):
            pos = 0
            sp = close
            print("Selling now at "+str(sp))
            pc = (sp/bp - 1)*100
            percentchange.append(pc)
            
    if(num==df["Adj Close"].count() - 1 and pos==1):
        pos = 0
        sp = close
        print("Selling now at "+str(sp))
        pc = (sp/bp - 1)*100
        percentchange.append(pc)
    
    num += 1 
print(percentchange)

Buying now at 1878.050048828125
Selling now at 2146.449951171875
[14.291413719843483]


# Analysing results

## Calculating parameters

In [66]:
gains = 0
ng = 0
losses = 0
nl = 0
totalR = 1

for i in percentchange:
    if(i>0):
        gains += i
        ng += 1
    else:
        losses += i
        nl += 1
        
    totalR = totalR*((i/100)+1)

totalR = round((totalR-1)*100, 2)

if(ng > 0):
    avgGain = gains/ng
    maxR = str(max(percentchange))
else:
    avgGain = 0
    maxR = "undefined"
    
if(nl > 0):
    avgLoss = losses/nl
    maxL = str(min(percentchange))
    ratio = (-avgGain/avgLoss)
else:
    avgLoss = 0
    maxL = "undefined"
    ratio = "inf"
    
if(ng>0 or nl>0):
    battingAvg = ng/(ng+nl)
else:
    battingAvg=0

## Summary chart

In [67]:
print()
print("Results for "+ ticker +" going back to "+str(df.index[0])+", Sample size: "+str(ng+nl)+" trades")
print("EMAs used: "+str(emasUsed))
print("Batting Avg: "+ str(battingAvg))
print("Gain/loss ratio: "+ ratio)
print("Average Gain: "+ str(avgGain))
print("Average Loss: "+ str(avgLoss))
print("Max Return: "+ maxR)
print("Max Loss: "+ maxL)
print("Total return over "+str(ng+nl)+ " trades: "+ str(totalR)+"%" )
#print("Example return Simulating "+str(n)+ " trades: "+ str(nReturn)+"%" )
print()


Results for RELIANCE.NS going back to 2020-07-09 00:00:00, Sample size: 1 trades
EMAs used: [3, 5, 8, 10, 12, 15, 30, 35, 40, 45, 50, 60]
Batting Avg: 1.0
Gain/loss ratio: inf
Average Gain: 14.291413719843483
Average Loss: 0
Max Return: 14.291413719843483
Max Loss: undefined
Total return over 1 trades: 14.29%

